# Порядки на мономах с доказательствами

## Лексикографический порядок (lex)

- Мономы сравниваются по первой отличающейся переменной слева направо. 
- Если в первой отличающейся позиции показатель степени у монома $a$ больше, чем у $b$, то $a \succ b$.

---
1.  **Если редукция привела к 0**

На каждом шаге редукции мы вычитаем элемент идеала, и если в итоге получили ноль, исходный полином представляется как комбинация порождающих, то есть принадлежит идеалу:
$$
f = \sum (\text{элементы из } I) = 0 \implies f \in I
$$

2.

# Любой ли набор полиномов — базис Грёбнера?

**Вопрос:**  
Всегда ли $\operatorname{in}(I) = \langle \operatorname{in}(g_1), \ldots, \operatorname{in}(g_m) \rangle ?$

**Ответ:**  
Нет

**Контрпример:**  
$$
I = \langle x^2 - y, \, xy - x \rangle, \quad f = y^2 - y \in I
$$

Ведущие члены порождающих:
$$
\operatorname{in}(x^2 - y) = x^2, \quad \operatorname{in}(xy - x) = xy
$$

Получаем идеал
$$
\langle x^2, xy \rangle. 
$$

Но $y^2 \in \operatorname{in}(I)$, так как $f \in I$, $\operatorname{in}(f) = y^2$, а $y^2 \notin \langle x^2, xy \rangle$.

# Редуцированный базис Грёбнера

## Определение и свойства

Редуцированный базис Грёбнера — это базис Грёбнера, в котором:

- Все ведущие коэффициенты равны 1;
- Ведущие мономы попарно не делятся друг на друга;
- Каждый ненулевой моном любого базисного элемента не делится на ведущие мономы остальных;
- Никакой элемент нельзя дополнительно редуцировать по остальным. 

---

## Утверждение

- Любой базис Грёбнера можно привести к редуцированному.

In [21]:
from fractions import Fraction
from collections import defaultdict
from typing import Optional

Monomial = tuple[int, ...]
PolyDict = dict[Monomial, Fraction]

class RingElement:
    """
    Представление полинома в K[x_1, ..., x_n]
    через словарь: моном (кортеж степеней) -> коэффициент
    """

    def __init__(self, poly_dict: Optional[PolyDict] = None, order: str = 'lex') -> None:
        """
        poly_dict: словарь {(a1, a2, ..., an): коэффициент}
        order: тип мономиального порядка ('lex', 'grlex')
        """
        self.poly: PolyDict = defaultdict(Fraction)
        if poly_dict:
            for monom, coeff in poly_dict.items():
                if coeff != 0:
                    self.poly[tuple(monom)] = Fraction(coeff)
        self.order: str = order
        self._remove_zeros()

    def _remove_zeros(self) -> None:
        """Удалить нулевые коэффициенты"""
        self.poly = {m: c for m, c in self.poly.items() if c != 0}

    def _compare_monomials(self, m1: Monomial, m2: Monomial) -> bool:
        """
        Сравнить два монома согласно выбранному порядку.
        Возвращает: True если m1 > m2, False если m1 <= m2
        """
        if self.order == 'lex':
            return m1 > m2
        elif self.order == 'grlex':
            deg1 = sum(m1)
            deg2 = sum(m2)
            if deg1 != deg2:
                return deg1 > deg2
            return m1 > m2
        else:
            raise ValueError(f"Unknown order: {self.order}")

    def leading_monomial(self) -> Optional[Monomial]:
        """Возвращает ведущий моном"""
        if not self.poly:
            return None
        if self.order == 'grlex':
            return max(self.poly.keys(), key=lambda m: (sum(m), m))
        else:
            return max(self.poly.keys())

    def leading_coefficient(self) -> Fraction:
        """Возвращает ведущий коэффициент"""
        lm = self.leading_monomial()
        if lm is None:
            return Fraction(0)
        return self.poly[lm]

    def leading_term(self) -> Optional[tuple[Monomial, Fraction]]:
        """Возвращает ведущий член (моном * коэффициент)"""
        lm = self.leading_monomial()
        if lm is None:
            return None
        return (lm, self.poly[lm])

    def is_zero(self) -> bool:
        """Проверить, является ли полином нулевым"""
        return len(self.poly) == 0

    def normalize(self) -> 'RingElement':
        """Нормализовать полином так, чтобы ведущий коэффициент = 1"""
        lc = self.leading_coefficient()
        if lc != 0:
            normalized_dict: PolyDict = {m: c / lc for m, c in self.poly.items()}
            return RingElement(normalized_dict, self.order)
        return RingElement({}, self.order)

    def __add__(self, other: 'RingElement') -> 'RingElement':
        """Сложение полиномов"""
        result: defaultdict[Monomial, Fraction] = defaultdict(Fraction)
        for m, c in self.poly.items():
            result[m] += c
        for m, c in other.poly.items():
            result[m] += c
        return RingElement(dict(result), self.order)

    def __sub__(self, other: 'RingElement') -> 'RingElement':
        """Вычитание полиномов"""
        result: defaultdict[Monomial, Fraction] = defaultdict(Fraction)
        for m, c in self.poly.items():
            result[m] += c
        for m, c in other.poly.items():
            result[m] -= c
        return RingElement(dict(result), self.order)

    def __mul__(self, other: 'RingElement | int | Fraction') -> 'RingElement':
        """Умножение полиномов"""
        if isinstance(other, (int, Fraction)):
            result: PolyDict = {m: c * other for m, c in self.poly.items()}
            return RingElement(result, self.order)

        result: defaultdict[Monomial, Fraction] = defaultdict(Fraction)
        for m1, c1 in self.poly.items():
            for m2, c2 in other.poly.items():
                new_monom = tuple(a + b for a, b in zip(m1, m2))
                result[new_monom] += c1 * c2
        return RingElement(dict(result), self.order)

    def __rmul__(self, scalar: int | Fraction) -> 'RingElement':
        """Умножение на скаляр слева"""
        return self.__mul__(scalar)

    def __repr__(self) -> str:
        """Красивый вывод полинома"""
        if not self.poly:
            return "0"

        terms: list[str] = []
        for monom in sorted(self.poly.keys(), reverse=True):
            c = self.poly[monom]
            if all(x == 0 for x in monom):
                terms.append(f"{c}")
            else:
                var_part = "".join(
                    f"x{i}^{e}" if e > 1 else f"x{i}" if e == 1 else ""
                    for i, e in enumerate(monom)
                )
                if c == 1:
                    terms.append(var_part)
                elif c == -1:
                    terms.append(f"-{var_part}")
                else:
                    terms.append(f"{c}*{var_part}")

        return " + ".join(terms).replace(" + -", " - ")

    def __eq__(self, other: object) -> bool:
        """Проверка равенства полиномов"""
        if not isinstance(other, RingElement):
            return False
        return self.poly == other.poly

In [22]:
def gcd_monomial(m1: Monomial, m2: Monomial) -> Monomial:
    """Вычислить НОД двух мономов"""
    return tuple(min(a, b) for a, b in zip(m1, m2))


def lcm_monomial(m1: Monomial, m2: Monomial) -> Monomial:
    """Вычислить НОК двух мономов"""
    return tuple(max(a, b) for a, b in zip(m1, m2))


def divide_monomial(m1: Monomial, m2: Monomial) -> Optional[Monomial]:
    """Разделить моном m1 на m2, вернуть m1/m2 или None если не делится"""
    result: list[int] = []

    for a, b in zip(m1, m2):
        if a < b:
            return None
        result.append(a - b)

    return tuple(result)


def normal_form(f: 'RingElement', G: list['RingElement']) -> 'RingElement':
    """
    Редуцировать полином f по базису G.
    Возвращает нормальную форму (остаток после всех возможных редукций).
    """
    f_copy = RingElement(dict(f.poly), f.order)

    while not f_copy.is_zero():
        lm_f = f_copy.leading_monomial()
        lc_f = f_copy.leading_coefficient()

        found = False
        for g in G:
            lm_g = g.leading_monomial()
            if lm_g is None:
                continue

            quotient = divide_monomial(lm_f, lm_g)
            if quotient is not None:
                lc_g = g.leading_coefficient()

                factor_coeff = lc_f / lc_g
                factor_poly = RingElement({quotient: factor_coeff}, f.order)

                reduction_term = factor_poly * g
                f_copy = f_copy - reduction_term
                found = True
                break

        if not found:
            break

    return f_copy


def S_polynomial(f: 'RingElement', g: 'RingElement') -> 'RingElement':
    """
    Вычислить S-полином двух полиномов f и g.
    S(f, g) = lcm(LM(f), LM(g)) / LT(f) * f - lcm(LM(f), LM(g)) / LT(g) * g
    """
    if f.is_zero() or g.is_zero():
        return RingElement({}, f.order)
    lm_f = f.leading_monomial()
    lm_g = g.leading_monomial()

    lc_f = f.leading_coefficient()
    lc_g = g.leading_coefficient()
    lcm = lcm_monomial(lm_f, lm_g)

    factor_f = divide_monomial(lcm, lm_f)
    factor_g = divide_monomial(lcm, lm_g)

    f_factor = RingElement({factor_f: Fraction(1) / lc_f}, f.order)
    g_factor = RingElement({factor_g: Fraction(1) / lc_g}, f.order)

    s_poly = f_factor * f - g_factor * g

    return s_poly


def gcd_monomial_coeff(m1: Monomial, m2: Monomial) -> bool:
    """Проверить, взаимно просты ли мономы (НОД = 1)"""
    gcd = gcd_monomial(m1, m2)
    return all(x == 0 for x in gcd)

In [23]:
def buchberger(F: list['RingElement'], order: str = 'lex') -> list['RingElement']:
    """
    Алгоритм Бухбергера для построения базиса Грёбнера.
    F: список полиномов (RingElement)
    order: мономиальный порядок

    Возвращает редуцированный базис Грёбнера.
    """

    G: list['RingElement'] = [f.normalize() for f in F if not f.is_zero()]

    pairs_to_check: list[tuple[int, int]] = [(i, j) for i in range(len(G)) for j in range(i + 1, len(G))]

    iteration: int = 0

    while pairs_to_check:
        iteration += 1
        print(f"Итерация {iteration}: {len(pairs_to_check)} пар для проверки, |G| = {len(G)}")

        i, j = pairs_to_check.pop(0)

        f = G[i]
        g = G[j]

        lm_f = f.leading_monomial()
        lm_g = g.leading_monomial()

        if gcd_monomial_coeff(lm_f, lm_g):
            continue

        s = S_polynomial(f, g)

        s_reduced = normal_form(s, G)

        if not s_reduced.is_zero():
            s_reduced_norm = s_reduced.normalize()

            new_pairs = [(len(G), k) for k in range(len(G))]
            pairs_to_check.extend(new_pairs)

            G.append(s_reduced_norm)
            print(f"  Добавлен новый элемент в G. Новый размер: {len(G)}")

    print(f"\nАлгоритм завершён после {iteration} итераций.")
    print(f"Финальный размер базиса: {len(G)}")

    G_minimal: list['RingElement'] = []
    for i, g in enumerate(G):
        is_redundant = False
        for j, h in enumerate(G):
            if i != j:
                lm_h = h.leading_monomial()
                lm_g = g.leading_monomial()
                if divide_monomial(lm_g, lm_h) is not None:
                    is_redundant = True
                    break
        if not is_redundant:
            G_minimal.append(g)

    print(f"После минимизации: {len(G_minimal)} элементов\n")

    return G_minimal


def is_in_ideal(f: 'RingElement', G: list['RingElement']) -> bool:
    """
    Проверить, принадлежит ли полином f идеалу, порождённому G.
    Использует редукцию по нормализованному редуцированному базису.

    Возвращает True если f in I, иначе False.
    """
    G_norm: list['RingElement'] = [g.normalize() for g in G]

    f_reduced: 'RingElement' = normal_form(f, G_norm)

    return f_reduced.is_zero()

In [24]:
print("=" * 66)
print("ТЕСТ 1: Один полином")
print("=" * 66)

f1 = RingElement({(2, 0): 1, (0, 1): -1}, order='lex')  # x^2 - y
print(f"f1 = {f1}")

G1 = buchberger([f1], order='lex')
print(f"Базис Грёбнера:")
for g in G1:
    print(f"  {g}")

print("\n" + "=" * 66)
print("ТЕСТ 2: Два полинома")
print("=" * 66)

f2a = RingElement({(2, 0): 1, (0, 1): -1}, order='lex')
f2b = RingElement({(1, 1): 1, (1, 0): -1}, order='lex')

print(f"f1 = {f2a}")
print(f"f2 = {f2b}")

G2 = buchberger([f2a, f2b], order='lex')
print(f"\nБазис Грёбнера:")
for i, g in enumerate(G2):
    print(f"  g{i+1} = {g}")

test_poly = RingElement({(0, 2): 1, (0, 1): -1}, order='lex')
print(f"\nПроверка: y^2 - y ∈ I ?")
print(f"Результат: {is_in_ideal(test_poly, G2)}")

print("\n" + "=" * 66)
print("ТЕСТ 3: Сложная система (степень 3)")
print("=" * 66)

f3a = RingElement({(3, 0): 1, (1, 1): -1}, order='grlex')
f3b = RingElement({(0, 3): 1, (1, 1): -1}, order='grlex')

print(f"f1 = {f3a}")
print(f"f2 = {f3b}")

G3 = buchberger([f3a, f3b], order='grlex')
print(f"\nБазис Грёбнера (порядок grlex):")
for i, g in enumerate(G3):
    print(f"  g{i+1} = {g}")

ТЕСТ 1: Один полином
f1 = x0^2 - x1

Алгоритм завершён после 0 итераций.
Финальный размер базиса: 1
После минимизации: 1 элементов

Базис Грёбнера:
  x0^2 - x1

ТЕСТ 2: Два полинома
f1 = x0^2 - x1
f2 = x0x1 - x0
Итерация 1: 1 пар для проверки, |G| = 2
  Добавлен новый элемент в G. Новый размер: 3
Итерация 2: 2 пар для проверки, |G| = 3
Итерация 3: 1 пар для проверки, |G| = 3

Алгоритм завершён после 3 итераций.
Финальный размер базиса: 3
После минимизации: 3 элементов


Базис Грёбнера:
  g1 = x0^2 - x1
  g2 = x0x1 - x0
  g3 = x1^2 - x1

Проверка: y^2 - y ∈ I ?
Результат: True

ТЕСТ 3: Сложная система (степень 3)
f1 = x0^3 - x0x1
f2 = -x0x1 + x1^3
Итерация 1: 1 пар для проверки, |G| = 2

Алгоритм завершён после 1 итераций.
Финальный размер базиса: 2
После минимизации: 2 элементов


Базис Грёбнера (порядок grlex):
  g1 = x0^3 - x0x1
  g2 = -x0x1 + x1^3


### Система 1: Две параболы

**Уравнения:**
$$
\begin{cases}
x^2 + y - 1 = 0 \\
x + y^2 - 1 = 0
\end{cases}
$$

#### Шаг 1: Выбор порядка мономов

**Выбор:** Лексикографический порядок $x > y$. 

**Обоснование:** Этот порядок исключает переменные последовательно, начиная с $x$.

In [25]:
eq1 = RingElement({(2, 0): 1, (0, 1): 1, (0, 0): -1}, order='lex')
eq2 = RingElement({(1, 0): 1, (0, 2): 1, (0, 0): -1}, order='lex')

print(f"Уравнение 1: x² + y - 1 = 0")
print(f"Уравнение 2: x + y² - 1 = 0")

G_sys1 = buchberger([eq1, eq2], order='lex')

print("\nБазис Грёбнера:")
for i, g in enumerate(G_sys1):
  print(f" g{i+1} = {g}")

Уравнение 1: x² + y - 1 = 0
Уравнение 2: x + y² - 1 = 0
Итерация 1: 1 пар для проверки, |G| = 2
  Добавлен новый элемент в G. Новый размер: 3
Итерация 2: 2 пар для проверки, |G| = 3
Итерация 3: 1 пар для проверки, |G| = 3

Алгоритм завершён после 3 итераций.
Финальный размер базиса: 3
После минимизации: 2 элементов


Базис Грёбнера:
 g1 = x0 + x1^2 - 1
 g2 = x1^4 - 2*x1^2 + x1


#### Шаг 3: Исключение переменных

Построив базис Грёбнера для системы
$$
\begin{cases}
x^2 + y - 1 = 0 \\
x + y^2 - 1 = 0
\end{cases}
$$
с лексикографическим порядком $x > y$, получаем следующий базис:
$$
\left\{  
\begin{array}{l}
g_1(x, y) = x + y^2 - 1 \\
g_2(y) = y^4 - 2y^2 + y
\end{array}
\right. 
$$

Последний элемент $g_2(y)$ зависит только от $y$, поэтому все корни исходной системы находятся через последовательное исключение переменных:

1.

In [26]:
from sympy import symbols, solve, groebner
from sympy.polys.polytools import Poly

x, y = symbols('x y')
eq1_sym = x ** 2 + y - 1
eq2_sym = x + y ** 2 - 1

solutions = solve([eq1_sym, eq2_sym], [x, y])
print(f"\nРешения системы: {solutions}")

G_sympy = groebner([eq1_sym, eq2_sym], x, y, order='lex')
print(f"\nБазис Грёбнера (SymPy): {G_sympy}")


Решения системы: [(0, 1), (1, 0), (-(-3/2 - sqrt(5)/2)*(1/2 - sqrt(5)/2), -sqrt(5)/2 - 1/2), (-(-3/2 + sqrt(5)/2)*(1/2 + sqrt(5)/2), -1/2 + sqrt(5)/2)]

Базис Грёбнера (SymPy): GroebnerBasis([x + y**2 - 1, y**4 - 2*y**2 + y], x, y, domain='ZZ', order='lex')


---

### Система 2: Окружность и гипербола

**Уравнения:**
$$
\begin{cases}
x^2 + y^2 - 4 = 0 \\
xy - 1 = 0
\end{cases}
$$

#### Шаг 1: Выбор порядка мономов

**Выбор:** Порядок по степени с лексикографическим разрешением конфликтов (grlex), $x > y$. 

**Обоснование:** grlex часто быстрее для вычисления базиса при высоких степенях и хорошо подходит для систем с полиномами разных степеней.

In [27]:
eq1 = RingElement({(2, 0): 1, (0, 2): 1, (0, 0): -4}, order='grlex')
eq2 = RingElement({(1, 1): 1, (0, 0): -1}, order='grlex')

print("Уравнение 1: x² + y² - 4 = 0")
print("Уравнение 2: xy - 1 = 0")

G_sys2 = buchberger([eq1, eq2], order='grlex')

print("\nБазис Грёбнера:")
for i, g in enumerate(G_sys2):
  print(f" g{i+1} = {g}")

from sympy import symbols, solve
x, y = symbols('x y')
eq1_sym = x ** 2 + y ** 2 - 4
eq2_sym = x*y - 1

solutions = solve([eq1_sym, eq2_sym], [x, y])
print(f"\nРешения (SymPy): {solutions}")

Уравнение 1: x² + y² - 4 = 0
Уравнение 2: xy - 1 = 0
Итерация 1: 1 пар для проверки, |G| = 2
  Добавлен новый элемент в G. Новый размер: 3
Итерация 2: 2 пар для проверки, |G| = 3
Итерация 3: 1 пар для проверки, |G| = 3

Алгоритм завершён после 3 итераций.
Финальный размер базиса: 3
После минимизации: 3 элементов


Базис Грёбнера:
 g1 = x0^2 + x1^2 - 4
 g2 = x0x1 - 1
 g3 = x0 + x1^3 - 4*x1

Решения (SymPy): [((-2 - sqrt(2 - sqrt(3)))*sqrt(2 - sqrt(3))*(2 - sqrt(2 - sqrt(3))), -sqrt(2 - sqrt(3))), (-(-2 + sqrt(2 - sqrt(3)))*sqrt(2 - sqrt(3))*(sqrt(2 - sqrt(3)) + 2), sqrt(2 - sqrt(3))), ((-2 - sqrt(sqrt(3) + 2))*(2 - sqrt(sqrt(3) + 2))*sqrt(sqrt(3) + 2), -sqrt(sqrt(3) + 2)), (-(-2 + sqrt(sqrt(3) + 2))*sqrt(sqrt(3) + 2)*(sqrt(sqrt(3) + 2) + 2), sqrt(sqrt(3) + 2))]


Базис Грёбнера (grlex, $x > y$):

$$
\left\{
\begin{array}{l}
g_1(x, y) = x^2 + y^2 - 4 \\
g_2(x, y) = x y - 1 \\
g_3(x, y) = x + y^3 - 4y
\end{array}
\right. 
$$

Последний элемент $g_3$ зависит только от $x$ и $y$:
$$
g_3(x, y) = x + y^3 - 4y = 0 \implies x = 4y - y^3
$$

Из $g_2(x, y) = x y - 1 = 0$ получаем $x = \frac{1}{y}$ (при $y \neq 0$).

---

### Система 3: Три переменные

**Уравнения:**
$$
\begin{cases}
x^2 + y^2 + z^2 - 1 = 0 \\
x^2 - y + z = 0 \\
x + y^2 - z^2 = 0
\end{cases}
$$

#### Особенности

При трёх переменных система становится значительно сложнее.  Базис Грёбнера может содержать много элементов.

In [28]:
eq1 = RingElement({(2, 0, 0): 1, (0, 2, 0): 1, (0, 0, 2): 1, (0, 0, 0): -1}, order='lex')
eq2 = RingElement({(2, 0, 0): 1, (0, 1, 0): -1, (0, 0, 1): 1}, order='lex')
eq3 = RingElement({(1, 0, 0): 1, (0, 2, 0): 1, (0, 0, 2): -1}, order='lex')

print("Построение базиса Грёбнера для системы 3x3...")
G_sys3 = buchberger([eq1, eq2, eq3], order='lex')

print(f"\nРазмер базиса: {len(G_sys3)}")
for i, g in enumerate(G_sys3):
  print(f" g{i+1} = {g}")

from sympy import symbols, solve, groebner

x, y, z = symbols('x y z')

eq1 = x**2 + y**2 + z**2 - 1
eq2 = x**2 - y + z
eq3 = x + y**2 - z**2

basis = groebner([eq1, eq2, eq3], x, y, z, order='lex')
print("\nБазис Грёбнера в SymPy:")
for i, g in enumerate(basis, 1):
    print(f"g{i}:", g)

last_poly = list(basis)[-1]
print("\nПоследний полином (только по z):")
print(last_poly)

roots_z = solve(last_poly, z)
print("\nПервые 4 корня по z:")
print(roots_z[:4], " ... ")

g_y = list(basis)[-2]
print("\nПолином для y (от y, z):")
print(g_y)

if roots_z:
    y_val = solve(g_y.subs(z, roots_z[0]), y)
    print("\nПервое значение y для первого корня z:")
    print(y_val)

g_x = list(basis)[-3]
print("\nПолином для x (от x, y, z):")
print(g_x)

Построение базиса Грёбнера для системы 3x3...
Итерация 1: 3 пар для проверки, |G| = 3
  Добавлен новый элемент в G. Новый размер: 4
Итерация 2: 5 пар для проверки, |G| = 4
  Добавлен новый элемент в G. Новый размер: 5
Итерация 3: 8 пар для проверки, |G| = 5
Итерация 4: 7 пар для проверки, |G| = 5
Итерация 5: 6 пар для проверки, |G| = 5
Итерация 6: 5 пар для проверки, |G| = 5
Итерация 7: 4 пар для проверки, |G| = 5
Итерация 8: 3 пар для проверки, |G| = 5
Итерация 9: 2 пар для проверки, |G| = 5
Итерация 10: 1 пар для проверки, |G| = 5
  Добавлен новый элемент в G. Новый размер: 6
Итерация 11: 5 пар для проверки, |G| = 6
Итерация 12: 4 пар для проверки, |G| = 6
Итерация 13: 3 пар для проверки, |G| = 6
Итерация 14: 2 пар для проверки, |G| = 6
  Добавлен новый элемент в G. Новый размер: 7
Итерация 15: 7 пар для проверки, |G| = 7
  Добавлен новый элемент в G. Новый размер: 8
Итерация 16: 13 пар для проверки, |G| = 8
Итерация 17: 12 пар для проверки, |G| = 8
Итерация 18: 11 пар для проверки, 

*Построенный базис:*

$$
\begin{cases}
g_1(x, y, z) = x + y^2 - z^2 \\
g_2(z) = z^8 - 2z^7 - z^6 + \frac{7}{2}z^5 - \frac{1}{4}z^4 - \frac{7}{4}z^3 + \frac{3}{4}z^2 + \frac{1}{4}z - \frac{1}{4} \\
g_3(y, z) = y - 12z^7 + 32z^6 - 12z^5 - 30z^4 + 25z^3 + z^2 - 9z + 3
\end{cases}
$$

*Последовательность исключения переменных:*

1.  Решаем $g_2(z) = 0$ — полином восьмой степени по $z$.

---

## Шаг 5(общий): Оценка вычеслительной сложности и оптимизации

### Сложность алгоритма Бухбергера

**Худший случай:** Двойная экспонента по числу переменных для некоторых классов систем. 

**Типичный случай:** Полиномиальная, но очень высокая степень.